# Librairies nécessaires

In [1]:
import numpy as np
import pandas as pd
import commentjson
import mlflow
from mlflow.tracking import MlflowClient
import shap
import os
import sys
import importlib
import generer_rapports as gr
from docx2pdf import convert

# Librairies personnelles
path = os.getcwd()
path_src = os.path.abspath(os.path.join(path, os.pardir,"src"))
sys.path.append(path_src)
#path_mode = os.path.abspath(os.path.join(path, os.pardir,"src","modelisation"))
#sys.path.append(path_mode)

from modelisation import mlflow_functions
importlib.reload(mlflow_functions)
importlib.reload(gr)

ModuleNotFoundError: No module named 'docx2html'

## Récupération des données du run MLFLOW

In [21]:
### Paramétrisation ###########################################

with open("config.json", encoding='utf-8') as file:
    int_config = commentjson.load(file)

#experiment_name  = int_config['experiment']

experiment_name  = int_config['client']+'-'+int_config['site']+ "-" + int_config['depart'] + "-" + int_config['case_study']
###############################################################


###############################################################

rep_tracking_mlflow = "file:///"+int_config['dir_models']+int_config['client']+'/'+int_config['site']+ "/" + int_config['depart'] +'/'+'ARCTIFACTS'



#rep_tracking_mlflow = "file:///"+int_config['dir_models']+"MLFLOW"
mlflow.set_tracking_uri(rep_tracking_mlflow)

experiment = mlflow.get_experiment_by_name(experiment_name)
if not experiment:
    print("Experience mlflow non trouvée")

experiment = mlflow.get_experiment_by_name(experiment_name)

runs = MlflowClient().search_runs(
    experiment_ids = experiment.experiment_id,
    filter_string  = "attributes.`run_name` =" + "'" + int_config['run_name'] + "'"
)

if not runs:
    print("Nom de l'experience ou du run invalide")
else:
    dico_infos = mlflow_functions.Load_Information_For_Report(runs[0])

    C:\Users\33623\Modeling\packages\generer rapports modele\rapport modele.ipynb

## Génération du rapport

In [1]:
from docx import Document

In [27]:
importlib.reload(gr)
docx = gr.word_docx(dico_infos)
docx.save('Rapport_modélisation.docx')
convert('Rapport_modélisation.docx')

  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
with mlflow.start_run(run_id=runs[0].info.run_id) as run:
    mlflow.log_artifact('Rapport_modélisation.docx', artifact_path="Rapports")
    mlflow.log_artifact('Rapport_modélisation.pdf', artifact_path="Rapports")


mlflow.end_run()